# How to develop and debug a mixed authorized Polarion LiveDoc

Welcome to the capella2polarion notebook where local development and debugging 
of an MA Polarion LiveDoc is showcased. Mixed-authorized means, that we have
a live document with sections enhanced by generated content from the model sync
or directly from the model. The possibilities are endless.

For MA LiveDocs you will need a pre-existing document. In there we need to declare
sections for auto generated content. The macro can be copied from the documentation
of [Mixed Authority document rendering](https://dsd-dbs.github.io/capella-polarion/configuration/render_documents.html#mixed-authority-mode).

This notebook will show you the following:
- How to render and maintain auto generated sections in a pre-existing Polarion livedoc
- How to configure the `capella2polarion_document_config.yaml.j2` for pipeline usage

As always before we can interact with the REST API of Polarion we need to prepare our
environment:
Inside an `.env` file there are the following values:
- POLARION_PROJECT
- POLARION_HOST
- POLARION_PAT
- MODEL_PATH

In [1]:
from capella2polarion.connectors import polarion_worker
from capella2polarion.converters import document_renderer, document_config

import dotenv
import os
import capellambse
import pathlib

dotenv.load_dotenv()

test_data_path = pathlib.Path("../../../tests/data")

model = capellambse.MelodyModel(os.environ.get(
    "MODEL_PATH") or str(test_data_path / "model/Melody Model Test.aird")
)
worker = polarion_worker.CapellaPolarionWorker(
    polarion_worker.PolarionWorkerParams(
            os.environ.get("POLARION_PROJECT") or "",
            os.environ.get("POLARION_HOST") or "",
            os.environ.get("POLARION_PAT") or "",
            delete_work_items=False,
        )
)
renderer = document_renderer.DocumentRenderer(
    worker.polarion_data_repo,
    model,
    os.environ.get("POLARION_PROJECT") or "",
    overwrite_heading_numbering=True,
    overwrite_layouts=True,
)

Don't forget to load the current work items from the sync into the worker, because we need them for references. If a work item isn't synced but a reference inserted in a live doc then the renderer will tell:
`Missing UUID: XXX-...` in the rendering. 

In [20]:
worker.load_polarion_work_item_map()

Now we load the current LiveDoc under the specific space `MA-Drafts`. This is configured in
the document config `ma_capella2polarion_document_config.yaml`:

In [2]:
document_rendering_config_path = pathlib.Path("configs/ma_capella2polarion_document_config.yaml.j2")
print(document_rendering_config_path.read_text(encoding="utf8"))

# Copyright DB InfraGO AG and contributors
# SPDX-License-Identifier: Apache-2.0

mixed_authority:
  - template_directory: document_templates
    heading_numbering: True
    project_id: SPT0Test240118
    work_item_layouts:
      _C2P_physicalLink:
        fields_at_start:
          - id
      _C2P_physicalActor:
        fields_at_start:
          - id
      _C2P_physicalComponentNode:
        fields_at_start:
          - id
      _C2P_physicalComponentBehavior:
        fields_at_start:
          - id
      _C2P_physicalFunction:
        fields_at_start:
          - id
      _C2P_physicalPort:
        fields_at_start:
          - id
    sections:
      PCDContent: test-pcd-mixed-auth.html.j2
    instances:
      - polarion_space: MA-Drafts
        polarion_name: Physical Components Description
        params:
          kinds:
            - SOFTWARE
            - HARDWARE



You can see that we even provide a `project_id`. So projects outside of the C2P sync project are targetable for enhancing MA Live Docs with generated sections. This only works if the PAT has access (REST-API) to this Polarion project. 

Now you can see that we have currently one section `PCDContent` configured and the only instance (target Live Doc) is `Physical System Spec - MA Showcase`.
Let's have a look into the template `test-pcd-mixed-auth.html.j2` for now:

## Polarion Jinja2 template for MA LiveDocs

In [22]:
print(pathlib.Path("document_templates/test-pcd-mixed-auth.html.j2").read_text(encoding="utf8"))

{#
    Copyright DB InfraGO AG and contributors
    SPDX-License-Identifier: Apache-2.0
#}

{% from 'common-macros.j2' import deployed_components %}
{% from 'polarion-props.j2' import table_attributes, th_attributes, td_attributes %}

{% for physical_component in model.search("PhysicalComponent") | unique(attribute="uuid") %}

    {{ heading(2, "Definition of " + physical_component.name, session) }}
    <workitem id="description-{{ loop.index }}">{{ physical_component.description | safe }}</workitem>

    {{ heading(3, "Component Boundary", session) }}
    <workitem id="context-{{ loop.index }}">
        The figure below provides an overview of <i>{{ physical_component.name }}</i>'s boundary and depicts all the nodes it is connected to.
    </workitem>
    {{ insert_work_item(physical_component, session) }}

    {% for kind in kinds %}
        {{ deployed_components(physical_component, kind, "Deployed " + kind.capitalize(), 3, session) }}
    {% endfor %}

{% endfor %}



We import the macro `deployed_components` from the common-macros Jinja2 template. This is a way to keep your templates short and precise. It definitely makes sense to divide the concerns of function declarations and template logic. Additional quirks are also imported for Polarion handling the tables correctly.
Then for now we have a simple logic implemented for our `PhysicalComponent`s. For each component we define the following:

- h2 with the name of the component included
- a text work item including the component's description
- h3 and a work item reference to the current component
- another for loop displaying subcomponents of a certain kind (we only allow SOFTWARE and HARDWARE from the config)

Emphasize on some details that seem minor but break the rendering:
```html
<workitem id="description-{{ loop.index }}">
```
IDs of work items in a Live-Doc need to be unique. With this statement the renderer will create a new work item of type text and the id set here. With `loop.index` we get the ID unique.

Pretty simple. Let's see the code for the macro and then we render this in Polarion:

In [24]:
print(pathlib.Path("document_templates/common-macros.j2").read_text(encoding="utf8"))

{#
    Copyright DB InfraGO AG and contributors
    SPDX-License-Identifier: Apache-2.0
#}

{% macro deployed_components(pc, kind, title, level, session) %}
    {% set components = [] %}
    {% for comp in pc.components %}
        {% if comp.kind == kind %}
            {% set _ = components.append(comp) %}
        {% endif %}
    {% endfor %}

    {{ heading(level, title, session) }}
    {% if components %}
        <workitem id="deployed-components-{{ pc.uuid }}-{{ kind }}">
            {{- pc.name }} has the following {{ kind.lower() }} components deployed:
        </workitem>
        {% for component in components %}
            {{ insert_work_item(component, session) }}
        {% endfor %}
    {% else %}
        <workitem id="deployed-components-{{ pc.uuid }}-{{ kind }}">
            <span style="color: red;"><b>{{- pc.name }}</b> has no subcomponents of kind {{ kind.lower() }}.</span>
        </workitem>
    {% endif %}
{% endmacro %}



Same here again:
```html
<workitem id="deployed-components-{{ pc.uuid }}-{{ kind }}">
```
With a clever combination of the parameters we get the ID unique. Think about how the macro gets called and adjust the template code accordingly.

Now let's look at what we need to do for preparing the Live Doc in Polarion:

![MA live doc initial](./_static/ma-live-doc.png)

The wiki macros you can copy from the documentation. It should look like this
```html
<div class="c2pAreaStart" id="PCDContent">
#set($statusList = ["doc_open", "draft", "planned", "inReview"])
#if ($statusList.contains($document.getStatus().id))
<p style="font-weight: bold;background-color: #FFFF00;text-align: center;">
DON'T REMOVE THIS<br>
↓↓↓Below this point all content is autogenerated and will be overwritten↓↓↓
</p>
#end
</div>
```

It is important that the ID matches with the one set in the config and also the class is `c2pAreaStart` for the section initializer and `c2pAreaEnd` for the section end.

If you want to know more about the features and limitations, head into the
documentation of the configuration for [MA live doc rendering](https://dsd-dbs.github.io/capella-polarion/features/render_documents.html#mixed-authority-mode).

## How to load the current Polarion LiveDoc and update it
Just the same way like we did in the FA LiveDoc showcase:

In [21]:
with document_rendering_config_path.open("r", encoding="utf8") as file:
    configs = document_config.read_config_file(file, model)

documents = worker.load_polarion_documents(configs.iterate_documents())
projects_document_data = renderer.render_documents(configs, documents)
for project, project_data in projects_document_data.items():
    worker.create_documents(project_data.new_docs, project)
    worker.update_documents(project_data.updated_docs, project)

This brings us to:

![MA live doc rendered once](./_static/ma-rendered-live-doc.png)

We should enable `context_diagrams` for the referenced work items:

In [25]:
document_rendering_config_path = pathlib.Path("configs/ma_capella2polarion_document_config.yaml.j2")
print(document_rendering_config_path.read_text(encoding="utf8"))

# Copyright DB InfraGO AG and contributors
# SPDX-License-Identifier: Apache-2.0

mixed_authority:
  - template_directory: document_templates
    heading_numbering: True
    project_id: SPT0Test240118
    work_item_layouts:
      _C2P_physicalLink:
        fields_at_start:
          - id
      _C2P_physicalActor:
        fields_at_start:
          - id
      _C2P_physicalComponentNode:
        fields_at_start:
          - id
        fields_at_end:
          - context_diagram
      _C2P_physicalComponentBehavior:
        fields_at_start:
          - id
        fields_at_end:
          - context_diagram
      _C2P_physicalFunction:
        fields_at_start:
          - id
        fields_at_end:
          - context_diagram
      _C2P_physicalPort:
        fields_at_start:
          - id
    sections:
      PCDContent: test-pcd-mixed-auth.html.j2
    instances:
      - polarion_space: MA-Drafts
        polarion_name: Physical Components Description
        params:
          kinds:
         

In [26]:
with document_rendering_config_path.open("r", encoding="utf8") as file:
    configs = document_config.read_config_file(file, model)

documents = worker.load_polarion_documents(configs.iterate_documents())
projects_document_data = renderer.render_documents(configs, documents)
for project, project_data in projects_document_data.items():
    worker.create_documents(project_data.new_docs, project)
    worker.update_documents(project_data.updated_docs, project)

This brings us:

![MA live doc rendered 2nd](./_static/ma-rendered-live-doc-last.png)

Be advanteagous and define new sections. Iterate over other attributes of the capellambse object and use the [documentation of py-capellambse](https://dsd-dbs.github.io/py-capellambse/code/capellambse.metamodel.html#). 